In [36]:
import numpy as np
import json
import glob
import re

# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
from nltk.corpus import stopwords

# vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [37]:
# preparing the data

def data_length(file):
    # global jsonLength
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # jsonLength = len(data)
    return len(data)


def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data


In [38]:
length = data_length('C:/Hammad Aslam/BS IT (post ADP)/3rd Semester/Capstone Project/Project/backend/datasets/categories/allFiles/computers_Laptops.json')
data_array = []
for i in range(0, 1):
    data = load_data(
        'C:/Hammad Aslam/BS IT (post ADP)/3rd Semester/Capstone Project/Project/backend/datasets/categories/allFiles/computers_Laptops.json')[i]["reviews"]
    # print(data[0]["review_body"])
    # print(data[i]["review_body"])
    data_array.append(data[0]["review_body"])
    # print(data)

# print(data_array)

In [39]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    text_out = []
    # a = texts.split()
    # b = [x for x in a if x!='']
    for text in texts:
        new_record = re.sub(' +', ' ', text)
        # print(new_record)
        doc = nlp(new_record)
        # print(doc)
        newText = []
        for token in doc:
            # if token.pos_ in allowed_postags:
            newText.append(token.lemma_)
            # token_list = [token for token in doc]
            filtered_list = [token for token in doc if not token.is_stop]
            lemmas = [
                f"{token.lemma_}"
                for token in filtered_list
            ]
            # string = ' '.join(lemmas)
        final = " ".join(lemmas)
        text_out.append(final)
    return text_out



In [40]:
lemmatizedText = lemmatization(data_array)

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final


new_data = gen_words(lemmatizedText)

In [41]:
id2word = corpora.Dictionary(new_data)

corpus = []
for text in new_data:
    a = id2word.doc2bow(text)
    corpus.append(a)

print(corpus[1])

IndexError: list index out of range

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus, id2word=id2word, num_topics=30, random_state=100, update_every=1, chunksize=100, passes=10, alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

c:\Users\GS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.272306  0.106868       1        1  11.995769
7      0.119778 -0.247974       2        1  11.446197
27     0.245441  0.149154       3        1  10.239274
2      0.155820  0.241778       4        1   9.777379
0      0.245274 -0.170178       5        1   7.236211
23    -0.024897 -0.305277       6        1   6.177527
12    -0.094764  0.293071       7        1   5.589691
11     0.283478  0.032882       8        1   5.410134
22    -0.239987 -0.137211       9        1   4.126536
15    -0.265773 -0.008675      10        1   4.084332
24    -0.134783  0.213683      11        1   3.463376
4     -0.050206 -0.211672      12        1   3.267183
1     -0.152872 -0.190002      13        1   3.184960
21     0.181600  0.022270      14        1   2.469805
26     0.041944  0.203353      15        1   2.424908
14     0.176349 -0.082421      16        1   2.382446
25     0.061269 -0.141423      17        1   1.775640
13    -0.142579  0.081553      18        1   1.545584
19    -0.094384 -0.059439      19        1   1.174875
8      0.012359  0.076335      20        1   0.663623
10     0.046529  0.046000      21        1   0.617444
6     -0.032400  0.052841      22        1   0.479588
16    -0.021280 -0.000944      23        1   0.159248
9     -0.006260  0.005232      24        1   0.045004
20    -0.006354  0.005045      25        1   0.044529
29    -0.005917  0.004884      26        1   0.043921
17    -0.006165  0.005022      27        1   0.043915
5     -0.006096  0.005028      28        1   0.043915
28    -0.006371  0.005097      29        1   0.043833
18    -0.006447  0.005119      30        1   0.043149, topic_info=         Term       Freq      Total Category  logprob  loglift
101      year  14.000000  14.000000  Default  30.0000  30.0000
30   computer  21.000000  21.000000  Default  29.0000  29.0000
45       asus  30.000000  30.000000  Default  28.0000  28.0000
7        good  25.000000  25.000000  Default  27.0000  27.0000
75    battery  19.000000  19.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
11      paper   0.001053   2.709450  Topic30  -6.7848  -0.1046
12      photo   0.001053   1.928968  Topic30  -6.7848   0.2352
14      price   0.001053  11.492746  Topic30  -6.7848  -1.5496
15    problem   0.001053   5.625963  Topic30  -6.7848  -0.8352
17    quality   0.001053   3.779731  Topic30  -6.7848  -0.4375

[1705 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
615       4  0.262505         able
615       5  0.262505         able
615       7  0.525009         able
205       4  0.591815   absolutely
706       2  0.877829  accelerator
...     ...       ...          ...
101      10  0.068135         year
101      12  0.340676         year
101      13  0.068135         year
101      14  0.068135         year
686      17  0.934501           yr

[1341 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 28, 3, 1, 24, 13, 12, 23, 16, 25, 5, 2, 22, 27, 15, 26, 14, 20, 9, 11, 7, 17, 10, 21, 30, 18, 6, 29, 19])